Dostrajanie modelu językowego GPT-2 w celu porównania wyników z eksperymentem strojenia soft-promptu. Oparte na: https://gist.github.com/mf1024/3df214d2f17f3dcc56450ddf0d5a4cd7

#Importy

Instalacja i import niezbędnych bibliotek.

In [ ]:
# Install transformers library.
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.8 MB/s eta 0:00:00


In [ ]:
import io
import os
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
#from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW,
                          get_linear_schedule_with_warmup,
                          GPT2LMHeadModel)

In [ ]:
import torch.nn as nn

#Weights & Biases

Instalowanie Weights & Biases

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=05fd920f2a75dca18c5ebf861d7da3489318fb484c6344cf6933d4eae6eb8e73
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import wandb

In [ ]:
wandb.login()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import os

#Ścieżki danych

Zdefiniowanie ścieżek danych.

In [ ]:
TRAIN_DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/PW PROJEKT/csvs/dets/raw_data/output"

In [ ]:
TRAIN_DATA_FILE = "train_data.csv"

In [ ]:
VAL_DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/PW PROJEKT/csvs/dets/raw_data/output"

In [ ]:
VAL_DATA_FILE = "val_data.csv"

In [ ]:
SAMPLE_DATA_FILE = "train_data_sample.csv"

#Urządzenie

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [ ]:
device

'cuda'

#Parametry treningu

In [ ]:
BATCH_SIZE = 1
EPOCHS = 1
LEARNING_RATE = 1e-4
WARMUP_STEPS = 0
MAX_SEQ_LEN = 230

In [ ]:
run = wandb.init(
    #Projekt, w którym to uruchomienie będzie rejestrowane
    project="pw-gsn-project-gpt2lmhead-finetuning",
    #Śledzone hiperparametry
    config={
        "learning_rate": LEARNING_RATE,
        "epochs": EPOCHS

    })

wandb: Currently logged in as: zawemi. Use `wandb login --relogin` to force relogin


#Zbiór danych

Stworzenie klasy Dataset (PyTorch)

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd

from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd

class QSADataset(Dataset):
    def __init__(self, qsa_data_path = TRAIN_DATA_DIR, qsa_data_file = TRAIN_DATA_FILE):
        super().__init__()

        train_qsa_data_path = os.path.join(qsa_data_path, qsa_data_file)

        self.qsa_texts = []
        self.qsa_labels = []

        qsa_df = pd.read_csv(train_qsa_data_path)

        for idx, row in qsa_df.iterrows():
          qsa_str = f'Question: {row["question"]}; Sentence: {row["description"]}; Answer:'
          qsa_label = row["binary_label"]

          self.qsa_texts.append(qsa_str)
          self.qsa_labels.append(qsa_label)

    def __len__(self):
        return len(self.qsa_texts)

    def __getitem__(self, item):
        return self.qsa_texts[item], self.qsa_labels[item]

In [ ]:
qsa_dataset = QSADataset(qsa_data_path = TRAIN_DATA_DIR, qsa_data_file = TRAIN_DATA_FILE)
qsa_dataloader = DataLoader(qsa_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
val_qsa_dataset = QSADataset(qsa_data_path = VAL_DATA_DIR, qsa_data_file = VAL_DATA_FILE)
val_qsa_dataloader = DataLoader(val_qsa_dataset, batch_size=BATCH_SIZE, shuffle=False)

#Model

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

In [ ]:
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
for param in model.parameters():
  param.requires_grad = True

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

#Funkcje pomocnicze

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def validation(val_dataloader):
  sum_loss = 0.0
  batch_count = 0

  predicted_labels = []
  true_labels = []

  cross_entropy_loss = nn.CrossEntropyLoss()

  for qsa in val_dataloader:
    text = torch.tensor(tokenizer.encode(qsa[0][0])).unsqueeze(0).to(device)
    with torch.no_grad():
      label = tokenizer.encode(qsa[1][0])
      label_index = label[0]
      outputs = model(input_ids=text)
      softmax_logits = torch.softmax(outputs.logits, dim=-1)
      predicted_label = softmax_logits.detach().argmax(axis=-1).flatten().tolist()[-1]
      predicted_labels.append(predicted_label)
      loss = cross_entropy_loss(outputs.logits[:,-1,:], torch.cuda.LongTensor([label_index]))

    sum_loss += loss.clone().detach()
    batch_count += 1
    avg_loss = sum_loss/batch_count
    true_labels.append(label_index)

    accuracy = accuracy_score(true_labels, predicted_labels)

  return avg_loss, accuracy

#Trening

In [ ]:
VALID_EVERY = 3000

In [ ]:
FT_PATH = "/content/drive/MyDrive/Colab Notebooks/PW PROJEKT/csvs/dets/fine_tuning"

In [ ]:
sum_loss = 0.0
batch_count = 0

predicted_labels = []
true_labels = []

models_folder = "trained_models_lmhead"
cross_entropy_loss = nn.CrossEntropyLoss()

if not os.path.exists(models_folder):
    os.mkdir(models_folder)

for epoch in range(EPOCHS):

    print(f"EPOCH {epoch+1}/{EPOCHS} started" + '=' * 30)

    for qsa in tqdm(qsa_dataloader):
        text = torch.tensor(tokenizer.encode(qsa[0][0])).unsqueeze(0).to(device)
        label = tokenizer.encode(qsa[1][0])
        label_index = label[0]
        outputs = model(input_ids=text)
        softmax_logits = torch.softmax(outputs.logits, dim=-1)
        predicted_label = softmax_logits.detach().argmax(axis=-1).flatten().tolist()[-1]
        predicted_labels.append(predicted_label)

        loss = cross_entropy_loss(outputs.logits[:,-1,:], torch.cuda.LongTensor([label_index]))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        sum_loss += loss.clone().detach()
        batch_count += 1
        avg_loss = sum_loss/batch_count
        wandb.log({"Avg loss": avg_loss})
        true_labels.append(label_index)
        accuracy = accuracy_score(true_labels, predicted_labels)
        wandb.log({"Accuracy": accuracy})

        if batch_count % VALID_EVERY == 0:
          print(f"Avg loss: {sum_loss/batch_count}")
          print(f"Accuracy: {accuracy}")
          #VALIDATION
          val_avg_loss, val_accuracy = validation(val_qsa_dataloader)
          wandb.log({"Validation loss": val_avg_loss})
          wandb.log({"Validation accuracy": val_accuracy})
          print(f"Validation loss: {val_avg_loss}")
          print(f"Validation accuracy: {val_accuracy}")
          print("=" * 30)

torch.save(model.state_dict(), f"{FT_PATH}/model_ft_e{EPOCHS}_lr{LEARNING_RATE}.pt")

EPOCH 1/1 started==============================


  0%|          | 0/37981 [00:00<?, ?it/s]

Avg loss: 0.7585980296134949
Accuracy: 0.5553333333333333
Validation loss: 0.6898394823074341
Validation accuracy: 0.536
Avg loss: 0.691111147403717
Accuracy: 0.6215
Validation loss: 0.514286458492279
Validation accuracy: 0.76
Avg loss: 0.6474021673202515
Accuracy: 0.6654444444444444
Validation loss: 0.5118120908737183
Validation accuracy: 0.773
Avg loss: 0.6148828268051147
Accuracy: 0.6914166666666667
Validation loss: 0.48437657952308655
Validation accuracy: 0.759
Avg loss: 0.5943849682807922
Accuracy: 0.7044666666666667
Validation loss: 0.4647201895713806
Validation accuracy: 0.807
Avg loss: 0.5766301155090332
Accuracy: 0.7166111111111111
Validation loss: 0.4711659252643585
Validation accuracy: 0.801
Avg loss: 0.5653981566429138
Accuracy: 0.7259523809523809
Validation loss: 0.47127678990364075
Validation accuracy: 0.792
Avg loss: 0.5539075136184692
Accuracy: 0.7337083333333333
Validation loss: 0.42092546820640564
Validation accuracy: 0.827
Avg loss: 0.5429832935333252
Accuracy: 0.739

In [ ]:
print(f"Validation loss: {val_avg_loss}")
print(f"Validation accuracy: {val_accuracy}")

Validation loss: 0.41634830832481384
Validation accuracy: 0.827


#Walidacja

In [ ]:
PATH = "/content/drive/MyDrive/Colab Notebooks/PW PROJEKT/csvs/dets/fine_tuning/model_ft_e1_lr0.0001.pt"

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model.load_state_dict(torch.load(PATH))
model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
avg_loss, val_accuracy = validation(val_qsa_dataloader)
print(f"Average loss = {avg_loss}")
print(f"Validation accuracy = {val_accuracy}")

Average loss = 0.4160844385623932
Validation accuracy = 0.819
